<img src="./img/stock-analyzer-ai-architecture.png">

In [ ]:
# Libs install
!pip3 install yfinance==0.2.41
!pip3 install crewai==0.28.8
!pip3 install crewai-tools==0.8.3
!pip3 install langchain==0.1.20
!pip3 install langchain-openai==0.1.7
!pip3 install langchain-community==0.0.38
!pip3 install duckduckgo-search==5.3.0

In [ ]:
# Import libs
import json
import os
from datetime import datetime, timedelta

import yfinance as yf
from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [ ]:
# Creating Stock Analyzer AI Tool

def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-15", end="2024-08-15")
    return stock

yahoo_finance_tool = Tool(
    name = "Stock Analyzer AI",
    description = "Fetches stock prices for {ticket} from the last year about a specific company from Yahoo Finance API.",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [ ]:
# Testing the tool
response = yahoo_finance_tool.run("GOOGL")
print(response)

In [ ]:
# Importing LLM GPT of OpenAI
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
llm = ChatOpenAI(
    model="gpt-3.5-turbo"
    )

In [ ]:
stockPriceAnalystAgent = Agent(
    role = "Senior Stock Price Analyst",
    goal = "Find the {ticket} stock price and analyses trends",
    backstory = """
    You're a highly experienced in analyzing the price of a specific stock and make
    predictions about its future price.
    """,
    verbose = True,
    llm = llm,
    max_iter = 3,
    memory = True,
    allow_delegation = False,
    tools = [yahoo_finance_tool]
)

In [ ]:
getStockPrice = Task(
    description = "Analyze de stock {ticket} price history and create a trend analyses of up, down or sideways",
    agent = stockPriceAnalystAgent,
    expected_output = """
    Specify the current trend stock price - up, down or sideways.
     eg. stock = 'AAPL', price UP.
     """,
)

In [ ]:
# Creating News Stock Searching Tool

search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [ ]:
newsStockAnalystAgent = Agent(
    role = "Stock News Analyst",
    goal = """
    Create a short summary of the market news related to the stock {ticket} company.
    Specify the current trend - up, down or sideways with the news context. For each requested stock 
    asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.
    """,
    backstory = """
    You're a highly experienced in analyzing the market trends and news and have 
    tracked assets for more then 20 years.

    You're also master level analyst in the traditional markets and have deep understanding of human 
    psychology.
    
    You understand news, theirs tittles and theirs information, but you look at those with a health 
    dose of skepticism. You consider also the source of the news articles.
    """,
    verbose = True,
    llm = llm,
    max_iter = 7,
    memory = True,
    allow_delegation = False,
    tools = [search_tool]
)

In [ ]:
getStockNews = Task(
    description = f"""
    Take the stock and always include BTC to it (if no request).
    Use the search tool to search each one individually.
    
    The current date is {datetime.now().strftime('%d/%m/%Y')}.
    
    Compose the result into a helpful report.
    """,
    agent = newsStockAnalystAgent,
    expected_output = """
    A summary of the overall market and one sentence summary for each request 
    asset.
    
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED OF NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
)

In [ ]:
stockAnalystWriterAgent = Agent(
    role = "Senior Stock Analyst Writer",
    goal = """
    Analyse the trends price and news and write an insightful compelling and informative 3 paragraph long newsletter based on the 
    stock report and price trend.
    """,
    backstory = """
    You're widely accepted as the best stock analyst in the market. You understand 
    complex concepts and create compelling stories and narratives that resonate withe the audiences.
    
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multiple opinions when analyzing anything data.
    """,
    verbose = True,
    llm = llm,
    max_iter = 3,
    memory = True,
    allow_delegation = True
)

In [ ]:
writeAnalyses = Task(
    description = """
    Use the stock price trend and the stock news report to create an analyses and write the newsletter 
    about the {ticket} company that is brief and highlights the most important points.

    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?

    Include the previous analyses of stock trend and news summary.
    """,
    agent = stockAnalystWriterAgent,
    expected_output = """An eloquent 3 paragraphs newsletter format as markdown in a easy readable manner. 
    Its should contain:

    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest 
    - main part provides the meat of the analyses including the news summary and fear/greed score.
    - summary - key facts and concrete future trends prediction - up, down, sideways
    
    """,
    context = [getStockPrice, getStockNews]
)

In [ ]:
crew = Crew(
    agents = [stockPriceAnalystAgent, newsStockAnalystAgent, stockAnalystWriterAgent],
    tasks = [getStockPrice, getStockNews, writeAnalyses],
    verbose = 2,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 10
)

In [ ]:
results = crew.kickoff(inputs={"ticket": "GOOGL"})

In [ ]:
print("Result of the Senior Stock Price Analyst Agent:\n")
Markdown(results['tasks_outputs'][0].exported_output)

In [ ]:
print("Result of the Stock News Analyst Agent:\n")
Markdown(results['tasks_outputs'][1].exported_output)

In [ ]:
print("Result of the Senior Stock Analyst Writer:\n")
Markdown(results['tasks_outputs'][2].exported_output)

In [ ]:
print("Result of the Senior Stock Analyst Writer:\n")
Markdown(results["final_output"])